This imports all relevant libraries, including the requests library 

In [1]:
import records
import numpy as np
import pandas as pd
import toyplot
from sklearn import preprocessing

This statement creates an object using the records library to download records of the *Bombus* taxon

In [2]:
rec = records.Epochs("Bombus", 1900, 1960, 3, **{"country": "CA"})

This is a simplified view of the data frame of the record created above

In [3]:
rec.sdf.head()

,species,year,epoch,country,stateProvince
0,Bombus fervidus,1900,1900,Canada,Ontario
1,Bombus vagans,1900,1900,Canada,New Brunswick
2,Bombus borealis,1900,1900,Canada,Manitoba
3,Bombus borealis,1901,1900,Canada,Ontario
4,Bombus fervidus,1901,1900,Canada,British Columbia


In [4]:
rec.sdf.shape

(17742, 5)

2000 entries set aside for testing is about 10% of the total number of entries

In [122]:
tsize = 2000

In [123]:
#convert x vals to 2-d array
species = rec.df.species.values[:, None]
#convery y vals to 1-d array
state = rec.df.stateProvince.values

Since the data is categorical, I use LabelEncoder to assign each label a numerical value (ie, represent the data numerically)

In [126]:
label = preprocessing.LabelEncoder()
filledsp = rec.df["species"].fillna(0)
labeledsp = label.fit_transform(filledsp.astype(str))

filledst = rec.df["stateProvince"].fillna(0)
labeledst = label.fit_transform(filledst.astype(str))

In [127]:
#split data for training and testing

tsize = 200
x_test = labeledsp[:tsize]
x_train = labeledsp[tsize:]

y_test = labeledst[:tsize]
y_train = labeledst[tsize:]

In [128]:
from sklearn.neighbors import KNeighborsClassifier

In [129]:
#run the K Neartest Neighbors package using the training data
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train.reshape(-1, 1), y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [130]:
#get predictions of y values under the model using predit() on the x test values 
yfit = knn.predict(x_test.reshape(-1, 1))

In [131]:
#compare the predicted y values with the actual y values
np.mean(yfit == y_test)

0.22

This is pretty bad, which makes sense since this model is based on categorical data and makes very little sense

In [132]:
c, a, m = toyplot.scatterplot(
    labeledst, 
    labeledsp,
    size = 6,
    opacity=0.2,
    width=600, 
    height=500);

a.scatterplot(
    yfit,
    x_test,
    mstyle={"stroke": 'black'},
);


</

The main problem with this is the methods can work, but don't necessarily make sense. If the idea here is that if given a species, we can predict which province it's from, just looking at the data indicates that there isn't a model that has high power with such limited info since there is extensive overlap in the species present in certain provinces (which makes sense, since species don't know what province boundary lines are). All in all, I recognize that this model doesn't make sense with the data I chose. 